# Just enough LangChain for RAG

I use OpenAI below. You may need to adjust for a different LLM using examples from week 4.

## Setting up Elasticsearch 

In [ ]:
import os
from icecream import ic
from dotenv import load_dotenv
load_dotenv(".env", override=True)

from elasticsearch import Elasticsearch

es = None

if 'ELASTIC_CLOUD_ID' in os.environ:
  es = Elasticsearch(
    cloud_id=os.environ['ELASTIC_CLOUD_ID'],
    basic_auth=(os.environ['ELASTIC_USER'], os.environ['ELASTIC_PASSWORD']),
    request_timeout=30
  )
elif 'ELASTIC_URL' in os.environ:
  es = Elasticsearch(
    os.environ['ELASTIC_URL'],
    basic_auth=(os.environ['ELASTIC_USER'], os.environ['ELASTIC_PASSWORD']),
    request_timeout=30
  )
else:
  print("env needs to set either ELASTIC_CLOUD_ID or ELASTIC_URL")

if es:
    ic(es.info()['tagline']) # should return cluster info

## Using a langchain abstraction for ELSER

In [ ]:
from langchain_elasticsearch import ElasticsearchStore

index_name = "sotu_chunks_elser"

es_elser = ElasticsearchStore(
    es_connection=es,
    index_name=index_name,
    strategy=ElasticsearchStore.SparseVectorRetrievalStrategy(model_id=".elser_model_2_linux-x86_64")
)

## Example of a simple semantic search

In [ ]:
human_input = "what did the president say about the resolve of the Ukrainian people?"

best_result = ic(es_elser.similarity_search(human_input, k=1)[0])

## Getting the LLM ready

This is likely the part you'll have to change if you are not using OpenAI.  Use one of the ones you got working in week 4 and note you need the "Chat" model from Langchain for this to work well.

In [ ]:
import openai
from langchain_openai import ChatOpenAI
def load_openai_llm():
    openai.api_key = os.environ["OPENAI_API_KEY"]
    default_model = "gpt-3.5-turbo"
    llm = ChatOpenAI(
        temperature=0.3,
        model=default_model
    )
    return llm
LLM=load_openai_llm()


In [ ]:
LLM.invoke("The current prime minister of the United Kingdom in 2023 is ")

## LangChain PromptTemplate

LangChain prompts are templates in the following format, letting you bring your own variables retrieved from the structured and unstrutured documents we indexed into Elasticsearch

The following code composes a prompt that could be used against OpenAI

If you are using LLama2 or another LLM that requires special tokens to separate the system prompt, you'll need to add those yourself.

In [ ]:
TEMPLATE = """You are a helpful AI Chatbot that answers questions 
about past Presidential State of the Union addresses in short responses using only the following provided context.
If you can't answer the question using the following context say "I don't know".

Context: 
On {date}, President {administration} said the following:
{context}

Human: {input}
AI:"""


from langchain.prompts.prompt import PromptTemplate

PROMPT = PromptTemplate(
    input_variables=["date", "administration", "context", "input"], 
    template=TEMPLATE
)

human_input = "what did the president say about the resolve of the Ukrainian people?"

print(PROMPT.format(
    date= best_result.metadata["date"],
    administration= best_result.metadata["administration"],
    context= best_result.page_content,
    input= human_input
))

## LangChain ELSER retrieval with a filter
the chunks have metadata, a fitler will make retrieval more accurate. the chunks don't spell out which president is speaking in the text field, so mentions of the president by name in the question can throw off ELSER. ELSER may gravitate towards the introductions where the president mentions their spouse or a former president (with same last name) in the building etc.

In [ ]:
from langchain.chains import LLMChain

index_name = "sotu_chunks_elser"

es_elser = ElasticsearchStore(
    es_connection=es,
    index_name=index_name,
    strategy=ElasticsearchStore.SparseVectorRetrievalStrategy(model_id=".elser_model_2_linux-x86_64")
)

def whatDidPresSay(human_input, administration=None, k=1, vebose=False):

    filter = [{"term": {"metadata.administration.keyword": administration}}] if administration else None
    best_result = es_elser.similarity_search(human_input, filter = filter, k=k)

    return best_result

human_input = "what did the president say about the resolve of the Ukrainian people?"

best_result = whatDidPresSay(human_input=human_input, administration="Biden", vebose=True)

best_result[0]



## LangChain the legacy way
with LLMChain used by explicit function call

In [ ]:
from langchain.chains import LLMChain
from langchain.prompts.prompt import PromptTemplate

def retrieve_sotu_chain(question, administration):

    TEMPLATE = """You are a helpful AI Chatbot that answers questions 
    about past Presidential State of the Union addresses in short responses using only the following provided context.
    If you can't answer the question using the following context say "I don't know".

    Context: 
    On {date}, President {administration} said the following:
    {context}

    Human: {input}
    AI:"""

    PROMPT = PromptTemplate(
        input_variables=["date", "administration", "context", "input"], 
        template=TEMPLATE
    )

    conversation = LLMChain(
        prompt=PROMPT,
        llm=LLM,
        verbose=False
    )

    best_result = whatDidPresSay(human_input=question, administration=administration)[0]

    result = conversation.invoke({
        "input":question, 
        "administration":best_result.metadata["administration"], 
        "context":best_result.page_content, 
        "date":best_result.metadata['date'] 
    })
    return result


for pres in ['Biden', 'Trump', 'Obama', "Bush43", "Clinton"]:
    human_input = f"what did the president say about the economy and economic development?"
    result = retrieve_sotu_chain(question=human_input, administration=pres)["text"]
    print(f"{pres}: {result}")
    print("")


## LangChain with LCEL "LangChain Expression Language"
Because all tech needs a pipelined query language in 2024

In [ ]:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

TEMPLATE = """You are a helpful AI Chatbot that answers questions 
about past Presidential State of the Union addresses in short responses using only the following provided context.
If you can't answer the question using the following context say "I don't know".

Context: 
On {date}, President {administration} said the following:
{context}

Human: {input}
AI:"""

PROMPT = PromptTemplate(
        input_variables=["date", "administration", "context", "input"], 
        template=TEMPLATE
    )


chain = PROMPT | LLM | output_parser

for pres in ['Biden', 'Trump', 'Obama', "Bush43", "Clinton"]:
    question = "What did the president say about the economy and economic development?"
    best_results = whatDidPresSay(human_input=question, administration=pres)

    if best_results != []:
        best_result = best_results[0]
        
        response = chain.invoke({
                "input":question, 
                "administration":best_result.metadata["administration"], 
                "context":best_result.page_content, 
                "date":best_result.metadata['date'] 
            })
        print(f"{pres}: {response}")
        print("")
    

## Making the filter with AI



In [ ]:
from enum import Enum
from langchain.output_parsers.enum import EnumOutputParser

class Presidents(Enum):
    BIDEN = "Biden"
    TRUMP = "Trump"
    OBAMA = "Obama"
    BUSH = "Bush43"
    CLINTON = "Clinton"
    UNKNOWN = "Unknown"
parser = EnumOutputParser(enum=Presidents)

question = "What did president Trump say about the economy and economic development"

whichPresTemplate = PromptTemplate(
    input_variables=["input"],
    template="""Out of the following options, Which person is this question about? Answer Unknown if you are not sure
    Options: Biden, Trump, Obama, Bush43, Clinton
    Question: {input}"""
)

identifyPresChain = whichPresTemplate | LLM | parser

presEnum = identifyPresChain.invoke({"input":question})

presEnum.value


In [ ]:
TEMPLATE = """You are a helpful AI Chatbot that answers questions 
about past Presidential State of the Union addresses in short responses using only the following provided context.
If you can't answer the question using the following context say "I don't know".

Context: 
On {date}, President {administration} said the following:
{context}

Human: {input}
AI:"""

PROMPT = PromptTemplate(
        input_variables=["date", "administration", "context", "input"], 
        template=TEMPLATE
    )


chain = PROMPT | LLM | output_parser


question = f"What did the first black president say about the economy and economic development?"

presEnum = identifyPresChain.invoke({"input":question})

administration = None if presEnum.value == "Unknown" else presEnum.value
if administration:
    print(f"> Limiting search to the SOTU addresses of administration: {administration}")

best_results = whatDidPresSay(human_input=question, administration=administration)

if best_results != []:
    best_result = best_results[0]
    
    response = chain.invoke({
            "input":question, 
            "administration":best_result.metadata["administration"], 
            "context":best_result.page_content, 
            "date":best_result.metadata['date'] 
        })
    print(f"{administration}: {response}")
    print("")
    